In [1]:
import supervisely_lib as sly
import os
import subprocess

In [2]:
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
print("Server address: ", address)
print("Your API token: ", token)

Server address:  http://192.168.1.69:5555
Your API token:  OfaV5z24gEQ7ikv2DiVdYu1CXZhMavU7POtJw2iDtQtvGUux31DUyWTXW6mZ0wd3IRuXTNtMFS9pCggewQWRcqSTUi4EJXzly8kH7MJL1hm3uZeM2MCn5HaoEYwXejKT


In [4]:
api = sly.Api(address, token)

In [5]:
# get some context - team and workspace
team = api.team.get_list()[0]
workspace = api.workspace.get_list(team.id)[0]
print("Current context: Team {!r}, Workspace {!r}".format(team.name, workspace.name))

Current context: Team 'max', Workspace 'my_super_workspace_002'


In [6]:
# clone project from explore
project_name = 'lemons_annotated'
if api.project.exists(workspace.id, project_name):
    project_name = api.project.get_free_name(workspace.id, project_name)
task_id = api.project.clone_from_explore('Supervisely/Demo/lemons_annotated', workspace.id, project_name)
api.task.wait(task_id, api.task.Status.FINISHED)
project = api.project.get_info_by_name(workspace.id, project_name)
print("Project has been sucessfully cloned from explore: ")
print(project)

Project has been sucessfully cloned from explore: 
ProjectInfo(id=781, name='lemons_annotated_018', description='', size='861069', readme='', workspace_id=9, created_at='2019-03-13T17:26:49.699Z', updated_at='2019-03-13T17:26:49.699Z')


In [7]:
project_dir = '/workdir/downloaded_project'
sly.fs.remove_dir(project_dir)
project_fs = sly.Project(project_dir, sly.OpenMode.CREATE)

In [8]:
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)
project_fs.set_meta(meta)

In [9]:
for dataset in api.dataset.get_list(project.id):
    dataset_fs = project_fs.create_dataset(dataset.name)
    for image in api.image.get_list(dataset.id):
        ann_info = api.annotation.download(image.id)
        ann = sly.Annotation.from_json(ann_info.annotation, meta)
        img = api.image.download_np(image.id)
        dataset_fs.add_item_np(image.name + image.ext, img, ann)

In [11]:
print(subprocess.Popen("tree {}".format(project_dir), shell=True, stdout=subprocess.PIPE).stdout.read().decode('utf-8'))

/workdir/downloaded_project
├── ds1
│   ├── ann
│   │   ├── IMG_0748.jpeg.json
│   │   ├── IMG_1836.jpeg.json
│   │   ├── IMG_2084.jpeg.json
│   │   ├── IMG_3861.jpeg.json
│   │   ├── IMG_4451.jpeg.json
│   │   └── IMG_8144.jpeg.json
│   └── img
│       ├── IMG_0748.jpeg
│       ├── IMG_1836.jpeg
│       ├── IMG_2084.jpeg
│       ├── IMG_3861.jpeg
│       ├── IMG_4451.jpeg
│       └── IMG_8144.jpeg
└── meta.json

3 directories, 13 files

